In [ ]:
import json
import time

In [ ]:
class Base:
  def success(self, message, data=None):
    if data == None:
      return json.dumps({"status": "success", "message": message}, ensure_ascii=False)
    else:
      return json.dumps({"status": "success", "message": message, "data": data}, ensure_ascii=False)

  def error(self, message):
    return json.dumps({"status": "error", "message": message}, ensure_ascii=False)

In [ ]:
class ShoppingCart(Base):
  def __init__(self):
    self.shopping_carts = []

  def add_cart(self, member_id):
    self.cart_id = len(self.shopping_carts) + 1
    self.member_id = member_id
    self.create_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    data = {
      'cart_id': self.cart_id,
      'member_id': self.member_id,
      'create_time': self.create_time
    }
    self.shopping_carts.append(data)

  def get_carts(self):
    print(self.success("", self.shopping_carts))

# 測試
shopping_cart_instance = ShoppingCart()
shopping_cart_instance.add_cart(1)
shopping_cart_instance.add_cart(2)
shopping_cart_instance.add_cart(3)
shopping_cart_instance.get_carts()

{"status": "success", "message": "", "data": [{"cart_id": 1, "member_id": 1, "create_time": "2024-12-09 11:34:06"}, {"cart_id": 2, "member_id": 2, "create_time": "2024-12-09 11:34:06"}, {"cart_id": 3, "member_id": 3, "create_time": "2024-12-09 11:34:06"}]}


In [ ]:
class Product(Base):
  def __init__(self):
    self.products = []

  def add_product(self, product_serialize):
    try:
      self.product_id = len(self.products) + 1
      self.create_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
      product = json.loads(product_serialize)
      product['product_id'] = self.product_id
      product['create_time'] = self.create_time
      product['last_update_time'] = self.create_time
      self.products.append(product)
      print(self.success("產品新增成功", product))
    except json.JSONDecodeError:
      print(self.error("產品資料格式不正確"))

  def get_product(self, product_id, output=True):
    for product in self.products:
      if product['product_id'] == product_id:
        if output: print(self.success("", product))
        return product
    if output: print(self.error(f"沒找到產品編號 {product_id}"))
    return {}

# 測試
product_instance = Product()

# 新增產品測試
product_instance.add_product('{"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png"}')
product_instance.add_product('{"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png"}')

# 取得產品測試
product_instance.get_product(1)
product_instance.get_product(3)


{"status": "success", "message": "產品新增成功", "data": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-09 08:13:47", "last_update_time": "2024-12-09 08:13:47"}}
{"status": "success", "message": "產品新增成功", "data": {"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png", "product_id": 2, "create_time": "2024-12-09 08:13:47", "last_update_time": "2024-12-09 08:13:47"}}
{"status": "success", "message": "", "data": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-09 08:13:47", "last_update_time": "2024-12-09 08:13:47"}}
{"status": "error", "message": "

{}

In [ ]:
class ShoppingCartItem(Base):
  def __init__(self, product_instance):
    self.cart_items = {}
    self.cart_item_id = 0
    self.product_instance = product_instance

  def add_cart_item(self, cart_id, product_id, quantity=1):
    # 初始化該購物車的商品列表
    if cart_id not in self.cart_items:
        self.cart_items[cart_id] = []

    # 檢查購物車內是否已有相同產品
    for item in self.cart_items[cart_id]:
      if item['product_id'] == product_id:
        print(self.error("產品已存在於購物車"))
        return

    # 從 Product 實例取得產品資料
    product = self.product_instance.get_product(product_id, False)

    # 如果產品尚未加入購物車
    self.cart_item_id += 1
    self.create_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    new_item = {
      'cart_id': cart_id,
      'product_id': product_id,
      'cart_item_id': self.cart_item_id,
      'cart_item_serialize': product,
      'cart_item_price': product["product_price"],
      'cart_item_qty': quantity,
      'cart_item_screenshot_url': product["product_screenshot_url"],
      'cart_item_status': product["product_status"],
      'create_time': self.create_time,
      'last_update_time': self.create_time
    }
    self.cart_items[cart_id].append(new_item)
    print(self.success("產品已加入購物車", new_item))

  def remove_cart_item(self, cart_id, product_id):
    if cart_id in self.cart_items:
      filtered_items = [
        item for item in self.cart_items[cart_id] if item['product_id'] != product_id
      ]
      if len(filtered_items) < len(self.cart_items[cart_id]):
        self.cart_items[cart_id] = filtered_items
        print(self.success(f"產品編號 {product_id} 已從購物車編號 {cart_id} 移除"))
      else:
        print(self.error(f"購物車編號 {cart_id} 中找不到產品編號 {product_id}"))
    else:
      print(self.error(f"找不到購物車編號 {cart_id}"))

  def get_cart_item(self, cart_item_id):
    for cart_id, items in self.cart_items.items():
      for item in items:
        if item['cart_item_id'] == cart_item_id:
          print(self.success("找到購物車項目", item))
          return
    print(self.error(f"找不到購物車項目 {cart_item_id}"))

  def sync_cart_item_status(self, cart_id):
    if cart_id not in self.cart_items:
      print(self.error(f"找不到購物車編號 {cart_id}"))
      return

    updated_items = []
    for item in self.cart_items[cart_id]:
      # 取得最新的產品資料
      product = None
      for prod in self.product_instance.products:
        if prod["product_id"] == item["product_id"]:
          product = prod
          break

      item["cart_item_status"] = product.get("product_status")
      item["last_update_time"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
      updated_items.append(item)

    # 更新購物車的項目列表
    self.cart_items[cart_id] = updated_items
    print(self.success(f"購物車編號 {cart_id} 的所有商品狀態已同步", updated_items))

  def update_cart_item_qty(self, cart_item_id, cart_item_qty):
    # 確保數量有效
    if cart_item_qty <= 0:
      print(self.error("數量必須大於零"))
      return

    # 遍歷購物車查找目標項目
    for cart_id, items in self.cart_items.items():
      for item in items:
        if item['cart_item_id'] == cart_item_id:
          # 更新數量與最後更新時間
          item['cart_item_qty'] = cart_item_qty
          item['last_update_time'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
          print(self.success(f"購物車項目 {cart_item_id} 數量已更新為 {cart_item_qty}", item))
          return

    # 若找不到項目
    print(self.error(f"找不到購物車項目 {cart_item_id}"))

  def get_cart_items(self, cart_id):
    print(self.success("取得所有購物車項目", self.cart_items))

  def empty_cart_items(self, cart_id):
    if cart_id in self.cart_items:
      self.cart_items[cart_id] = []
      print(self.success(f"購物車編號 {cart_id} 的所有項目已清空"))
    else:
      print(self.error(f"找不到購物車編號 {cart_id}"))

# 測試
shopping_cart_item_instance = ShoppingCartItem(product_instance)
shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"], 1)
shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"], 2)
shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"]+1, 1)
time.sleep(1)
shopping_cart_item_instance.update_cart_item_qty(shopping_cart_item_instance.cart_items[shopping_cart_instance.shopping_carts[0]["cart_id"]][0]["cart_item_id"], 3)
shopping_cart_item_instance.get_cart_item(1)
shopping_cart_item_instance.sync_cart_item_status(1)
shopping_cart_item_instance.get_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])
shopping_cart_item_instance.empty_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])
shopping_cart_item_instance.get_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])


{"status": "success", "message": "產品已加入購物車", "data": {"cart_id": 1, "product_id": 1, "cart_item_id": 1, "cart_item_serialize": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-09 08:13:47", "last_update_time": "2024-12-09 08:13:47"}, "cart_item_price": 50000, "cart_item_qty": 1, "cart_item_screenshot_url": "/project/電視機.png", "cart_item_status": "active", "create_time": "2024-12-09 08:35:20", "last_update_time": "2024-12-09 08:35:20"}}
{"status": "error", "message": "產品已存在於購物車"}
{"status": "success", "message": "產品已加入購物車", "data": {"cart_id": 1, "product_id": 2, "cart_item_id": 2, "cart_item_serialize": {"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png", "product_id": 2, "create_time": "2024-12-09 08:1

In [ ]:
#Member Base
from datetime import datetime

class Member:
    members = []  # 用於存儲所有會員的資料
    histories = {}  # 用於存儲會員的歷史紀錄
    next_id = 1  # 用於生成唯一的會員ID

    def __init__(self):
        self.member_user_id = None
        self.member_password = None
        self.member_email = None
        self.member_name = None

    @classmethod
    def is_user_registered(cls, member_user_id: str) -> bool:
        for member in cls.members:
            if member.member_user_id == member_user_id:
                return True
        return False

    def register(self, member_user_id: str, password: str, member_name: str, member_email: str) -> str:
        if not (member_user_id and password and member_name and member_email):
            return "註冊失敗，會員資料不完整"

        if Member.is_user_registered(member_user_id):
            return "註冊失敗，會員已註冊"

        self.member_user_id = member_user_id
        self.member_password = password
        self.member_email = member_email
        self.member_name = member_name

        Member.members.append(self)
        return f"{self.member_user_id} 成功加入會員"

    @classmethod
    def login(cls, member_user_id: str, password: str) -> str:
        for member in cls.members:
            if member.member_user_id == member_user_id and member.member_password == password:
                return "驗證成功"
        return "驗證失敗"

    @classmethod
    def add_member(cls, member_serialize: str) -> int:
        member_data = eval(member_serialize)
        member_id = cls.next_id
        cls.next_id += 1

        member = Member()
        member.member_user_id = member_data.get("member_user_id", "")
        member.member_password = member_data.get("member_password", "")
        member.member_email = member_data.get("member_email", "")
        member.member_name = member_data.get("member_name", "")
        cls.members.append(member)

        return member_id

    def update_member(self, member_id: int, member_serialize: str) -> str:
        if member_id > len(Member.members) or member_id <= 0:
            return "Member not found."

        member_data = eval(member_serialize)
        member = Member.members[member_id - 1]

        for key, value in member_data.items():
            if hasattr(member, key):
                setattr(member, key, value)

        return "Member updated successfully."

    @classmethod
    def reset_password(cls, member_user_id: str, password: str) -> str:
        for member in cls.members:
            if member.member_user_id == member_user_id:
                member.member_password = password
                return "密碼重設成功"
        return "會員未找到，重設失敗"

    @classmethod
    def add_history(cls, member_user_id: str, product_id: int) -> str:
        for member in cls.members:
            if member.member_user_id == member_user_id:
                if member_user_id not in cls.histories:
                    cls.histories[member_user_id] = []
                cls.histories[member_user_id].append({
                    "product_id": product_id,
                    "time": datetime.now(),
                })
                return "歷史紀錄新增成功"
        return "會員未找到，新增失敗"


# 測試案例
if __name__ == "__main__":
    # 測試註冊
    member1 = Member()
    print(member1.register("user001", "securepassword", "John Doe", "johndoe@example.com"))

    # 測試登入
    print(Member.login("user001", "securepassword"))

    # 測試新增會員
    new_member_data = '{"member_password": "123456", "member_user_id": "user002", "member_email": "user002@example.com", "member_name": "Jane Doe"}'
    print(Member.add_member(new_member_data))

    # 測試更新會員資料
    print(member1.update_member(1, '{"member_name": "John Smith"}'))

    # 測試重設密碼
    print(Member.reset_password("user001", "newpassword123"))

    # 測試新增歷史紀錄
    print(Member.add_history("user001", 101))

    # 測試註冊成功
    member_instance1 = Member()
    print("測試註冊成功:")
    registration_success1 = member_instance1.register(
        member_user_id="user123",
        password="securepassword",
        member_name="John Doe",
        member_email="johndoe@example.com"
    )
    print(registration_success1)  # Output: user123 成功加入會員

    # 測試註冊失敗（會員已存在）
    print("測試註冊失敗（會員已存在）:")
    member_instance2 = Member()
    registration_failure_duplicate = member_instance2.register(
        member_user_id="user123",
        password="anotherpassword",
        member_name="Jane Doe",
        member_email="janedoe@example.com"
    )
    print(registration_failure_duplicate)  # Output: 註冊失敗，會員已註冊

    # 測試登入成功
    print("測試登入成功:")
    login_success = Member.login(member_user_id="user123", password="securepassword")
    print(login_success)  # Output: 驗證成功

    # 測試登入失敗（密碼錯誤）
    print("測試登入失敗（密碼錯誤）:")
    login_failure_wrong_password = Member.login(member_user_id="user123", password="wrongpassword")
    print(login_failure_wrong_password)  # Output: 驗證失敗

    # 測試登入失敗（會員 ID 錯誤）
    print("測試登入失敗（會員 ID 錯誤）:")
    login_failure_wrong_user_id = Member.login(member_user_id="user456", password="securepassword")
    print(login_failure_wrong_user_id)  # Output: 驗證失敗

    # 測試註冊失敗（缺少必要參數）
    print("測試註冊失敗（缺少必要參數）:")
    member_instance3 = Member()
    registration_failure_missing_info = member_instance3.register(
        member_user_id="user456",
        password="anotherpassword",
        member_name="",  # 缺少名稱
        member_email="user456@example.com"
    )
    print(registration_failure_missing_info)  # Output: 註冊失敗，會員資料不完整


user001 成功加入會員
驗證成功
1
Member updated successfully.
密碼重設成功
歷史紀錄新增成功
測試註冊成功:
user123 成功加入會員
測試註冊失敗（會員已存在）:
註冊失敗，會員已註冊
測試登入成功:
驗證成功
測試登入失敗（密碼錯誤）:
驗證失敗
測試登入失敗（會員 ID 錯誤）:
驗證失敗
測試註冊失敗（缺少必要參數）:
註冊失敗，會員資料不完整


In [ ]:
#HistoryRecord Base
from datetime import datetime

class HistoryRecord:
    def __init__(self, member_id: int, product_id: int):
        self.member_id = member_id
        self.product_id = product_id
        self.create_time = datetime.now()

    def __repr__(self):
        return f"HistoryRecord(member_id={self.member_id}, product_id={self.product_id}, create_time={self.create_time})"


class HistoryManager:
    def __init__(self):
        self.history_records = []  # 存儲所有歷史記錄

    def add_history(self, member_id: int, product_id: int) -> str:
        record = HistoryRecord(member_id, product_id)
        self.history_records.append(record)
        return f"History added: {record}"

    def get_member_history(self, member_id: int):
        return [record for record in self.history_records if record.member_id == member_id]

    def __repr__(self):
        return f"HistoryManager(records={self.history_records})"

# 建立歷史管理實例
history_manager = HistoryManager()

# 新增歷史記錄
print(history_manager.add_history(1, 101))
print(history_manager.add_history(2, 102))
print(history_manager.add_history(1, 103))

# 獲取特定會員的歷史記錄
print("Member 1's History:", history_manager.get_member_history(1))
print("Member 2's History:", history_manager.get_member_history(2))

# 查看所有歷史記錄
print("All Records:", history_manager)



History added: HistoryRecord(member_id=1, product_id=101, create_time=2024-12-10 03:10:52.895214)
History added: HistoryRecord(member_id=2, product_id=102, create_time=2024-12-10 03:10:52.895375)
History added: HistoryRecord(member_id=1, product_id=103, create_time=2024-12-10 03:10:52.895469)
Member 1's History: [HistoryRecord(member_id=1, product_id=101, create_time=2024-12-10 03:10:52.895214), HistoryRecord(member_id=1, product_id=103, create_time=2024-12-10 03:10:52.895469)]
Member 2's History: [HistoryRecord(member_id=2, product_id=102, create_time=2024-12-10 03:10:52.895375)]
All Records: HistoryManager(records=[HistoryRecord(member_id=1, product_id=101, create_time=2024-12-10 03:10:52.895214), HistoryRecord(member_id=2, product_id=102, create_time=2024-12-10 03:10:52.895375), HistoryRecord(member_id=1, product_id=103, create_time=2024-12-10 03:10:52.895469)])
